In [28]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Reading the dataset

In [29]:
train_df = pd.read_csv('/kaggle/input/yelp-review-polarity/yelp_review_polarity_csv/yelp_review_polarity_csv/train.csv')
test_df  = pd.read_csv('/kaggle/input/yelp-review-polarity/yelp_review_polarity_csv/yelp_review_polarity_csv/test.csv')

In [30]:
train_df.columns = ['Rating','Review']
test_df.columns = ['Rating', 'Review']

In [31]:
train_df['Rating'] = train_df['Rating'].replace(2,1)
test_df['Rating'] = test_df['Rating'].replace(2,1)

In [32]:
train_df['Rating'] = train_df['Rating'].replace(1,0)
test_df['Rating'] = test_df['Rating'].replace(1,0)

Model

In [33]:
from transformers import BertTokenizer, TFBertForSequenceClassification
from transformers import InputExample, InputFeatures

model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased")
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

In [34]:
model.summary()

In [35]:
def convert_data_to_examples(train, test, DATA_COLUMN, LABEL_COLUMN): 
  train_InputExamples = train.apply(lambda x: InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this case
                                                          text_a = x[DATA_COLUMN], 
                                                          text_b = None,
                                                          label = x[LABEL_COLUMN]), axis = 1)

  validation_InputExamples = test.apply(lambda x: InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this case
                                                          text_a = x[DATA_COLUMN], 
                                                          text_b = None,
                                                          label = x[LABEL_COLUMN]), axis = 1)
  
  return train_InputExamples, validation_InputExamples

  train_InputExamples, validation_InputExamples = convert_data_to_examples(train, 
                                                                           test, 
                                                                           'DATA_COLUMN', 
                                                                           'LABEL_COLUMN')
  
def convert_examples_to_tf_dataset(examples, tokenizer, max_length=128):
    features = [] # -> will hold InputFeatures to be converted later

    for e in examples:
        # Documentation is really strong for this method, so please take a look at it
        input_dict = tokenizer.encode_plus(
            e.text_a,
            add_special_tokens=True,
            max_length=max_length, # truncates if len(s) > max_length
            return_token_type_ids=True,
            return_attention_mask=True,
            pad_to_max_length=True, # pads to the right by default # CHECK THIS for pad_to_max_length
            truncation=True
        )

        input_ids, token_type_ids, attention_mask = (input_dict["input_ids"],
            input_dict["token_type_ids"], input_dict['attention_mask'])

        features.append(
            InputFeatures(
                input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, label=e.label
            )
        )

    def gen():
        for f in features:
            yield (
                {
                    "input_ids": f.input_ids,
                    "attention_mask": f.attention_mask,
                    "token_type_ids": f.token_type_ids,
                },
                f.label,
            )

    return tf.data.Dataset.from_generator(
        gen,
        ({"input_ids": tf.int32, "attention_mask": tf.int32, "token_type_ids": tf.int32}, tf.int64),
        (
            {
                "input_ids": tf.TensorShape([None]),
                "attention_mask": tf.TensorShape([None]),
                "token_type_ids": tf.TensorShape([None]),
            },
            tf.TensorShape([]),
        ),
    )


DATA_COLUMN = 'DATA_COLUMN'
LABEL_COLUMN = 'LABEL_COLUMN'


In [39]:
train_df = train_df[0:100000]
test_df  = test_df[0:9000]

In [37]:
import tensorflow as tf

In [40]:
train_InputExamples, validation_InputExamples = convert_data_to_examples(train_df, test_df, 'Review', 'Rating')

train_data = convert_examples_to_tf_dataset(list(train_InputExamples), tokenizer)
train_data = train_data.shuffle(100).batch(32).repeat(2)

validation_data = convert_examples_to_tf_dataset(list(validation_InputExamples), tokenizer)
validation_data = validation_data.batch(32)

In [42]:
model.compile(optimizer='adam', loss="sparse_categorical_crossentropy", metrics= ['accuracy'])

In [43]:
model.fit(train_data, epochs=3, validation_data=validation_data)

In [21]:
import matplotlib.pyplot as plt

In [ ]:
metrics_df  = (pd.DataFrame({'loss':[2.0344e-04,1.1921e-07],'accuracy':[1.0000,1.0000],'val_loss':[1.1921e-07,1.1921e-07],'val_accuracy':[1.0000,1.000]}))

In [50]:
plt.plot(metrics_df['loss'], label = 'loss for Train set')
plt.plot(metrics_df['val_loss'], label = 'loss for Test Set')
plt.title('loss of our model')
plt.ylabel('loss')
plt.xlabel('epochs')
plt.legend()

In [51]:
plt.plot(metrics_df['accuracy'], label = 'accuracy for Train set')
plt.plot(metrics_df['val_accuracy'], label = 'accuracy for Test Set')
plt.title('accuracy of our model')
plt.ylabel('Accuracy')
plt.xlabel('epochs')
plt.legend()